# An example use case of stitching WFM data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dress import wfm

## Generate some fake data

In [ ]:
# Make a distribution using sine waves that resembles WFM experimental data
xmax = 7.5e4
nbins = 512
x = np.linspace(0., xmax, nbins)
dx = 9.5e3
y = np.abs(10.0 * np.sin((x-dx) * 3.5/ xmax) *
           np.sin((x-dx) * 27.5 / xmax)) + np.random.random(nbins)
y[:126] = 0.0
y[474:] = 0.0
# Add noise
y += 0.5 * np.random.random(nbins)

In [ ]:
# Plot distribution
fig1, ax1 = plt.subplots()
ax1.plot(x, y/np.sum(y))

Then we generate a random sample of ~100000 events using the above distribution as a probability density function

In [ ]:
nevents = 100000
events = np.random.choice(x, size=nevents, p=y/np.sum(y))
yy = (np.random.random(nevents) - 0.5) * 10.0

# Add fake Bragg peak
nbragg = nevents // 40
xbragg = np.random.choice(x, size=nbragg, p=y/np.sum(y))
ybragg = (xbragg + (np.random.random(nbragg) - 0.5)*1000.0) * (10.0 / xmax) - 5.0
events = np.concatenate([events, xbragg])
yy = np.concatenate([yy, ybragg])

In [ ]:
# Histogram and plot the event data
hist2d, ye, xe = np.histogram2d(yy, events, bins=256)
fig2, ax2 = plt.subplots(2, 1)
im = ax2[0].imshow(hist2d, aspect="auto", origin="lower", extent=[xe[0], xe[-1], ye[0], ye[-1]])
ax2[1].hist(events, bins=256)
fig2.colorbar(im, ax=ax2[0])

## Define the boundaries of the WFM frames

We begin by first loading the instrument setup

In [ ]:
# Get the V20 instrument chopper setup
v20setup = wfm.v20.setup()
# Get WFM frame parameters and generate TOF diagram
frames = wfm.get_frames(instrument=v20setup, plot=True)

Then we overlay the predicted locations of the frames onto the data

In [ ]:
# Plot the event data
fig3, ax3 = plt.subplots()
ax3.hist(events, bins=256)
for g in frames["gaps"]:
    ax3.axvline(x=g, color='r')

## Stitch the histogrammed data

In [ ]:
stitched = wfm.stitch(x=xe, y=hist2d, frames=frames)

In [ ]:
# Plot the stitched histogram
fig4, ax4 = plt.subplots(2, 1)
im4 = ax4[0].imshow(stitched, aspect="auto", origin="lower", extent=[xe[0], xe[-1], ye[0], ye[-1]])
ax4[1].plot(xe[:-1], np.sum(stitched, axis=0))
fig4.colorbar(im4, ax=ax4[0])